#Tuning Hyperparameters
##Ciaran Lowell
##Midterm Project

Unfortunately Professor Vasiliu, I wasn't able to debug this code in time. I'm just submitting what I had so far so I can get a low partial grade instead of a 0.

This project will demonstrate hyperparamaters and creation of polynomial features for the following regressors:

* Lasso

* Ridge

* Elastic Net

* SCAD

* Square Root Lasso 

* Stepwise

* Kernel

* Random Forest

* XGBoost




In [43]:
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
import keras
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
from yellowbrick.regressor import ResidualsPlot
import sklearn.datasets
import sklearn
import sklearn.datasets as dt
from sklearn.model_selection import GridSearchCV, cross_val_score 
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import yellowbrick as yb

In [44]:
#!pip install numpy==1.15.4

In [45]:
import numpy as np

In [46]:
def DoKFold(X,y,alpha):
  PE = []
  for idxtrain, idxtest in kf.split(X):
    X_train = X[idxtrain,:]
    y_train = y[idxtrain]
    X_test  = X[idxtest,:]
    y_test  = y[idxtest]
    model = sm.OLS(y_train,X_train)
    result = model.fit_regularized(method='sqrt_lasso', alpha=alpha)
    yhat_test = result.predict(X_test)
    PE.append(MAE(y_test,yhat_test))
  return np.mean(PE)

In [47]:
def cv_comparison(models, X, y, cv, model_names):
    # Initiate a DataFrame for the averages and a list for all measures
    cv_accuracies = pd.DataFrame()
    count = 0
    maes = []
    mses = []
    r2s = []
    accs = []
    # Loop through the models, run a CV, add the average scores to the DataFrame and the scores of 
    # all CVs to the list
    for model in models:
        mae = -np.round(cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv), 4)
        maes.append(mae)
        mae_avg = round(mae.mean(), 4)
        mse = -np.round(cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=cv), 4)
        mses.append(mse)
        mse_avg = round(mse.mean(), 4)
        r2 = np.round(cross_val_score(model, X, y, scoring='r2', cv=cv), 4)
        r2s.append(r2)
        r2_avg = round(r2.mean(), 4)
        acc = np.round((100 - (100 * (mae * len(X))) / y.sum()[0]), 4)
        accs.append(acc)
        acc_avg = round(acc.mean(), 4)
        cv_accuracies[model_names[count]] = [mae_avg, mse_avg, r2_avg, acc_avg]
        count = count + 1
    cv_accuracies.index = ['Mean Absolute Error', 'Mean Squared Error', 'R^2', 'Accuracy']
    return cv_accuracies, maes, mses, r2s, accs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/VasiliuLectures

## Dataset

This project will be analyzing the [Concrete Compressive Strength dataset](http://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength). This dataset provides 7 input variables (amounts of elements of different ingredients in the concrete, as well as Age) that contribute to the output variable, the concrete's compressive strength.

The dataset has 1030 rows. 

In [50]:
concreteDF = pd.read_csv('Concrete_DataSheet1.csv', header = 0, names = ['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer', 'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Concrete Compressive Strength'])

In [51]:
concreteDF

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete Compressive Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [52]:
concreteX = concreteDF.loc[:, concreteDF.columns != 'Concrete Compressive Strength']
concreteY = concreteDF.loc[:, concreteDF.columns == 'Concrete Compressive Strength']
concreteY


,Concrete Compressive Strength
0,79.99
1,61.89
2,40.27
3,41.05
4,44.30
...,...
1025,44.28
1026,31.18
1027,23.70
1028,32.77


## Data Preparation

Before we begin analyzing the data, we'll scale it by the z-scores. This is crucial for some of the regressors that we'll be implementing.

Rather than using a Train/Test split, we'll be evaluating all of our regressors via K-Fold analysis. 

In [53]:
scaler = sklearn.preprocessing.StandardScaler()
ConcreteX = scaler(ConcreteX)

## Creating Polynomial Features

The Concrete Compressive Strength dataset is a complex dataset, but several of the regressors we are fitting are linear regressions.

To improve their performance, before fitting them to the data, we will create polynomial features from the input data. 

I arbitrarily chose to make these features of the 3rd order.

In [54]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(3)
concreteXpoly = poly.fit_transform(concreteX)
concreteXpoly.shape

(1030, 165)

In [55]:
concreteY.dtypes

Concrete Compressive Strength    float64
dtype: object

## LASSO Regression

In [ ]:
lasso_alpha = [int(x) for x in np.linspace(1, 100, 100)]
lasso_params = {'alpha':lasso_alpha}
lasso = GridSearchCV(sklearn.linear_model.Lasso(), param_grid = lasso_params, cv = 5, verbose = 1)
lasso.fit(concreteXpoly, concreteY)
lassotuned = lasso.best_estimator_
lasso.best_params_

## Ridge Regression

Our hyperparameter *alpha* is tuned to be 100,000. This value will be stored for later comparison between the regressors.

In [59]:
ridge_alpha = [float(x) for x in np.linspace(30000, 100000, 1000)]
print(ridge_alpha)
ridge_params = {'alpha':ridge_alpha}
ridge = GridSearchCV(sklearn.linear_model.Ridge(), param_grid = ridge_params, cv = 5, verbose = 1)
ridge.fit(concreteXpoly, concreteY)
ridgetuned = ridge.best_estimator_
ridge.best_params_

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[30000.0, 30070.07007007007, 30140.14014014014, 30210.21021021021, 30280.28028028028, 30350.35035035035, 30420.42042042042, 30490.49049049049, 30560.56056056056, 30630.630630630632, 30700.7007007007, 30770.77077077077, 30840.84084084084, 30910.910910910912, 30980.98098098098, 31051.05105105105, 31121.12112112112, 31191.191191191192, 31261.26126126126, 31331.33133133133, 31401.401401401403, 31471.471471471472, 31541.54154154154, 31611.61161161161, 31681.681681681683, 31751.75175175175, 31821.82182182182, 31891.891891891893, 31961.961961961963, 32032.03203203203, 32102.1021021021, 32172.172172172173, 32242.242242242242, 32312.31231231231, 32382.382382382384, 32452.452452452453, 32522.522522522522, 32592.59259259259, 32662.662662662664, 32732.732732732733, 32802.802802802806, 32872.872872872875, 32942.942942942944, 33013.01301301301, 33083.08308308308, 33153.15315315315, 33223.22322322322, 33293.293293293296, 33363.363363363365, 33433.433433433434, 33503.5035035035, 33573.57357357357, 336

[Parallel(n_jobs=1)]: Done 5000 out of 5000 | elapsed:   57.9s finished


{'alpha': 100000.0}

## Elastic Net

In [60]:

eNet_alpha = [float(x) for x in np.linspace(.1, 100, 20)]
eNet_l1 = [float(x) for x in np.linspace(0.01, 1, 100)]
eNet_params = {'alpha':eNet_alpha, 'l1_ratio':eNet_l1}
eNet = GridSearchCV(sklearn.linear_model.ElasticNet(), param_grid = eNet_params, cv = 5, verbose = 0)
eNet.fit(concreteXpoly, concreteY)
eNettuned = eNet.best_estimator_
eNet.best_params_

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11011.1571758627, tolerance: 23.683356112220874
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12625.16220039179, tolerance: 23.73902658979369
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13749.371375192932, tolerance: 22.401226859563106
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increas

{'alpha': 0.1, 'l1_ratio': 0.01}

## SCAD

In [61]:

def scad_penalty(beta_hat, lambda_val, a_val):
    is_linear = (np.abs(beta_hat) <= lambda_val)
    is_quadratic = np.logical_and(lambda_val < np.abs(beta_hat), np.abs(beta_hat) <= a_val * lambda_val)
    is_constant = (a_val * lambda_val) < np.abs(beta_hat)
    
    linear_part = lambda_val * np.abs(beta_hat) * is_linear
    quadratic_part = (2 * a_val * lambda_val * np.abs(beta_hat) - beta_hat**2 - lambda_val**2) / (2 * (a_val - 1)) * is_quadratic
    constant_part = (lambda_val**2 * (a_val + 1)) / 2 * is_constant
    return linear_part + quadratic_part + constant_part
    
def scad_derivative(beta_hat, lambda_val, a_val):
    return lambda_val * ((beta_hat <= lambda_val) + (a_val * lambda_val - beta_hat)*((a_val * lambda_val - beta_hat) > 0) / ((a_val - 1) * lambda_val) * (beta_hat > lambda_val))

In [62]:
def scad_model(X,y,lam,a):
  n = X.shape[0]
  p = X.shape[1]
  # we add an extra column of 1 for the intercept
  X = np.c_[np.ones((n,1)),X]
  def scad(beta):
    beta = beta.flatten()
    beta = beta.reshape(-1,1)
    n = len(y)
    return 1/n*np.sum((y-X.dot(beta))**2) + np.sum(scad_penalty(beta,lam,a))
  
  def dscad(beta):
    beta = beta.flatten()
    beta = beta.reshape(-1,1)
    n = len(y)
    return np.array(-2/n*np.transpose(X).dot(y-X.dot(beta))+scad_derivative(beta,lam,a)).flatten()
  b0 = np.ones((p+1,1))
  output = minimize(scad, b0, method='L-BFGS-B', jac=dscad,options={'gtol': 1e-8, 'maxiter': 1e7,'maxls': 25,'disp': True})
  return output.x

In [63]:
def scad_predict(X,y,lam,a):
  beta_scad = scad_model(X,y,lam,a)
  n = X.shape[0]
  p = X.shape[1]
  # we add an extra column of 1 for the intercept
  X = np.c_[np.ones((n,1)),X]
  return X.dot(beta_scad)

In [64]:
def DoKFoldScad(X,y,lam,a):
  PE = []
  for idxtrain, idxtest in kf.split(X):
    X_train = X.iloc[idxtrain,:]
    y_train = y.iloc[idxtrain]
    X_test  = X.iloc[idxtest,:]
    y_test  = y.iloc[idxtest]
    beta_scad = scad_model(X_train,y_train,lam,a)
    n = X_test.shape[0]
    p = X_test.shape[1]
    # we add an extra columns of 1 for the intercept
    X1_test = np.c_[np.ones((n,1)),X_test]
    yhat_scad = X1_test.dot(beta_scad)
    PE.append(MAE(y_test,yhat_scad))
  return 1000*np.mean(PE)

In [65]:

def ScadCV(hparam):
  PEV = []
  for i in prange(len(hparam[:,0])):
    for j in prange(len(hparam[:,1])):
      PE  = []
      for idxtrain, idxtest in kf.split(X):
        X_train = X.iloc[idxtrain,:]
        y_train = y.iloc[idxtrain]
        X_test  = X.iloc[idxtest,:]
        y_test  = y.iloc[idxtest]
        beta_scad = scad_model(X_train,y_train,hparam[i,0],hparam[j,1])
        n = X_test.shape[0]
        p = X_test.shape[1]
        # we add an extra columns of 1 for the intercept
        X1_test = np.c_[np.ones((n,1)),X_test]
        yhat_scad = X1_test.dot(beta_scad)
        PE.append(MAE(y_test,yhat_scad))
    PEV.append(np.mean(PE))
  return np.array(PEV)

In [66]:
from numba import jit, prange
import scipy
from scipy.optimize import minimize
lam = 0.5
a = 1.001
p = concreteXpoly.shape[1]
b0 = np.random.normal(1,1,p)
y = concreteY
X = concreteX
#y = y.reshape((y.shape[0]), 1)

In [67]:
DoKFoldScad(X,y,0.1,1.2)

8316.480791060312

## Square Root Lasso

In [68]:
#! pip install --upgrade Cython
#! pip install --upgrade git+https://github.com/statsmodels/statsmodels
import statsmodels.api as sm
from numba import jit, prange
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.preprocessing import StandardScaler

In [69]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True,random_state=1234)

In [70]:
def DoKFoldSqrtLasso(X,y,alpha):
  PE = []
  for idxtrain, idxtest in kf.split(X):
    X_train = X.iloc[idxtrain,:]
    y_train = y.iloc[idxtrain]
    X_test  = X.iloc[idxtest,:]
    y_test  = y.iloc[idxtest]
    model = sm.OLS(y_train,X_train)
    result = model.fit_regularized(method='sqrt_lasso', alpha=alpha)
    yhat_test = result.predict(X_test)
    PE.append(MAE(y_test,yhat_test))
  return 1000*np.mean(PE)

In [71]:

def SqrtLassoCV(alpha):
  PEV = []
  print(X.shape)
  print(y.shape)
  for i in prange(len(alpha[:,0])):
    print("first loop")
    PE  = []
    a = alpha[i,0]
    print(a)
    for idxtrain, idxtest in kf.split(X):
      print("inner loop")
      print(idxtrain.shape)
      print(idxtest.shape)
      X_train = X[idxtrain,:]
      print("X_train.shape = ",X_train.shape)
      y_train = y[idxtrain]
      print("y_train.shape = ", y_train.shape)
      X_test  = X[idxtest,:]
      print("X_test.shape = ", X_test.shape)
      y_test  = y[idxtest]
      model = sm.OLS(y_train,X_train)
      result = model.fit_regularized(method='sqrt_lasso', alpha=a)
      yhat_test = result.predict(X_test)
      PE.append(MAE(y_test,yhat_test))
    PEV.append(np.mean(PE))
  return np.array(PEV)

In [72]:
def sqrtLasso(X,y,alpha, l1_wt):
    model = sm.OLS(y,X)
    result = model.fit_regularized(method='sqrt_lasso', alpha=alpha, l1_wt = l1_wt,profile_scale=True)
    return result.predict(X), result.params

In [73]:
! pip install pyswarms

In [74]:
import pyswarms as ps
alphas = np.arange(0,10,0.1)

In [75]:

x_max = 20 *np.ones(1)
x_min = 0.01 * np.ones(1)
bounds = (x_min, x_max)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=1, options=options, bounds=bounds)

cost, pos = optimizer.optimize(SqrtLassoCV, iters=50)

2021-05-18 02:20:47,587 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

pyswarms.single.global_best:   0%|          |0/50

(1030, 8)
(1030, 1)
first loop
16.728851836550753
inner loop
(824,)
(206,)


TypeError: ignored

## Stepwise Regression

In [76]:
# Implementation of stepwise regression
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

## Kernel Regression

## Random Forest

In [77]:
# Number of trees in Random Forest
rf_n_estimators = [int(x) for x in np.linspace(200, 1000, 5)]
rf_n_estimators.append(1500)

# Maximum number of levels in tree
rf_max_depth = [int(x) for x in np.linspace(5, 55, 11)]
# Add the default as a possible value
rf_max_depth.append(None)

# Number of features to consider at every split
rf_max_features = ['auto', 'sqrt', 'log2']

# Criterion to split on
rf_criterion = ['mse', 'mae']

# Minimum number of samples required to split a node
rf_min_samples_split = [int(x) for x in np.linspace(2, 10, 9)]

# Minimum decrease in impurity required for split to happen
rf_min_impurity_decrease = [0.0, 0.05, 0.1]

# Method of selecting samples for training each tree
rf_bootstrap = [True, False]

# Create the grid
rf_grid = {'n_estimators': rf_n_estimators,
               'max_depth': rf_max_depth,
               'max_features': rf_max_features,
               'criterion': rf_criterion,
               'min_samples_split': rf_min_samples_split,
               'min_impurity_decrease': rf_min_impurity_decrease,
               'bootstrap': rf_bootstrap}

In [78]:
from sklearn.model_selection import RandomizedSearchCV
rf_base = RandomForestRegressor()

# Create the random search Random Forest
rf_tune = RandomizedSearchCV(estimator = rf_base, param_distributions = rf_grid, n_iter = 50, cv = 5, verbose = 2,  
                               n_jobs = -1, random_state = 1693)

# Fit the random search model
rf_tune.fit(concreteX, concreteY)

# View the best parameters from the random search
rf_tune.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  8.2min finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': 30,
 'max_features': 'log2',
 'min_impurity_decrease': 0.0,
 'min_samples_split': 2,
 'n_estimators': 800}

In [79]:
rf_tuned = rf_tune.best_estimator_

## XGBoost

In [80]:
# eta (learning rate)
xgb_lr = [x for x in np.linspace(.01, .5, 100)]

#min_child_weight
xgb_mcw = [int(x) for x in np.linspace(1, 5, 5)]

# gamma (minimum loss reduction for split)
xgb_gamma = [int(x) for x in np.linspace(1,100, 10)]

# Create the grid
xgb_grid = {'eta': xgb_lr,
               'min_child_weight': xgb_mcw,
               'gamma': xgb_gamma}

In [81]:
xgb_base = xgb_reg
# Create the random search Random Forest
xgb_tuner = RandomizedSearchCV(estimator = xgb_base, param_distributions = xgb_grid, n_iter = 100, cv = 5, verbose = 1,  
                               n_jobs = -1, random_state = 1693)

In [82]:
xgb_tuner.fit(concreteX, concreteY)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:    6.8s


[02:40:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    9.0s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=1693, reg_alp...
                                                0.11393939393939392,
                                                0.11888888888888888,
                                                0.

In [83]:

# View the best parameters from the random search
xgb_tuned = xgb_tuner.best_estimator_
xgb_tuner.best_params_

{'eta': 0.039696969696969696, 'gamma': 78, 'min_child_weight': 3}

## Neural Network

In [87]:
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow import keras
modelnn = Sequential()
modelnn.add(Dense(128, activation="relu"))
modelnn.add(Dense(64, activation = "relu"))
modelnn.add(Dense(32, activation="relu"))
modelnn.add(Dense(8, activation="relu"))
modelnn.add(Dense(1, activation="linear"))

modelnn.compile(loss='mean_squared_error', optimizer=Adam(lr=1e-3, decay=1e-3 / 200), metrics = [keras.metrics.MeanAbsoluteError(), keras.metrics.MeanSquaredError(), keras.metrics.MeanAbsolutePercentageError()])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=1000)

history = modelnn.fit(concreteX, concreteY, validation_split=0.3, epochs=1000, batch_size=100, verbose=0, callbacks=[es])



In [88]:
modelnn.evaluate(concreteX, concreteY, verbose = 3)

[64.42028045654297, 5.135243892669678, 64.42028045654297, 19.638879776000977]

In [89]:
len(history.history['mean_squared_error'])

1000

In [92]:
np.mean(history.history['mean_absolute_error'])

4.228678412437439

In [93]:
sum(history.history['mean_squared_error'])/(len(history.history['mean_squared_error']))

49.42261221408844

In [111]:
nn = history
nncolumn = [ np.mean(nn.history['mean_absolute_error']), np.mean(nn.history['mean_squared_error']), "not calculated", np.mean(nn.history['mean_absolute_percentage_error'])]

In [ ]:

eNet = sklearn.linear_model.ElasticNet()
models = [mlr_reg, rf_reg, rf_tuned, xgb_reg, xgb_tuned, lasso_reg, lassotuned, ridge_reg, ridgetuned, eNet, eNettuned, nn]
model_names = ['Linear Regression', 'Random Forest', 'Random Forest (Tuned)', "XGBoost", "XGBoost Tuned", 'Lasso', 'Lasso (Tuned)', 'Ridge', 'Ridge (Tuned)', 'Elastic Net', 'Elastic Net (Tuned)']

# Run the Cross-Validation comparison with the models used in this analysis
comp, maes, mses, r2s, accs = cv_comparison(models, concreteX, concreteY, 5, model_names)
from tabulate import tabulate

print (tabulate(comp, headers= model_names))

In [96]:

# Create DataFrame for all R^2s
r2_comp = pd.DataFrame(r2s, index=comp.columns, columns=['1st Fold', '2nd Fold', '3rd Fold', 
                                                         '4th Fold', '5th Fold'])

# Add a column for the averages
r2_comp['Average'] = np.round(r2_comp.mean(axis=1),4)
r2_comp

,1st Fold,2nd Fold,3rd Fold,4th Fold,5th Fold,Average
Linear Regression,0.3302,0.4121,0.4518,0.6073,0.5037,0.4610
Random Forest,0.6720,0.6756,0.7347,0.8458,-1.3492,0.3158
Random Forest (Tuned),0.6912,0.7297,0.7712,0.8418,-0.9163,0.4235
XGBoost,0.7568,0.6630,0.7715,0.8838,-0.5973,0.4956
XGBoost Tuned,0.8008,0.6724,0.7732,0.8858,-0.4791,0.5306
Lasso,0.3614,0.4043,0.4464,0.6024,0.5122,0.4653
Lasso (Tuned),0.3749,0.2677,0.4014,0.4239,0.5700,0.4076
Ridge,0.3302,0.4121,0.4518,0.6072,0.5037,0.4610
Ridge (Tuned),0.4563,0.3620,0.4364,0.5604,0.5657,0.4762
Elastic Net,0.3547,0.4078,0.4488,0.6044,0.5105,0.4652


In [112]:

# Create DataFrame for all R^2s
df_comp = pd.DataFrame(comp, index=comp.index, columns=model_names)
df_comp["Neural Network"] = nncolumn

# Add a column for the averages
df_comp

,Linear Regression,Random Forest,Random Forest (Tuned),XGBoost,XGBoost Tuned,Lasso,Lasso (Tuned),Ridge,Ridge (Tuned),Elastic Net,Elastic Net (Tuned),Neural Network
Mean Absolute Error,23.3778,6.9017,7.3355,7.1851,7.1647,13.1123,12.9770,22.3185,16.6705,13.0158,12.7461,4.22868
Mean Squared Error,2993.9164,82.1897,93.7763,88.9623,86.2997,883.2568,868.7686,2812.5644,1465.9248,882.6379,862.2278,49.4226
R^2,-11.5798,0.5956,0.5067,0.5501,0.5713,-2.3052,-2.2222,-9.4351,-4.5580,-2.2897,-2.1987,not calculated
Accuracy,34.7316,80.7313,79.5199,79.9401,79.9969,63.3918,63.7694,37.6891,53.4578,63.6613,64.4141,14.7665
